In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import mysql.connector
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon,MultiPolygon,LineString,MultiLineString,LinearRing,Point
from shapely import symmetric_difference
from shapely.ops import nearest_points
from shapely.wkt import loads
import scipy
import time
import numpy as np
import random
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt
import osmnx as ox
import glob
from itertools import combinations,product
from copy import deepcopy
import os
from lib_contour_lines import *
import mysql.connector


In [2]:

cnx = mysql.connector.connect(user='spirz', 
                              password='this_is_my_PASSWORD_m8',
                              host='localhost',
                              database='dem_from_contours_lines',
                              autocommit=True)
cursor = cnx.cursor(buffered=True,dictionary=True)
contours_lines_table_name='contours_lines'
tree_edges_table_name="tree_edges"
osm_nodes_table_name="osm_nodes"
osm_intersections_table_name="osm_intersections"
2

2

In [3]:
t1=time.time()
G=build_graph(cursor)
t2=time.time()
print(t2-t1)

1.4359159469604492


In [4]:
t1=time.time()
cmd="SELECT osm_id,contour_line_id,ST_asText(geometry) AS geometry FROM %s WHERE contour_line_id IS NOT NULL"%osm_nodes_table_name
cursor.execute(cmd)
nodes_data=pd.DataFrame(cursor.fetchall())
nodes_data['geometry']=nodes_data['geometry'].apply(lambda pt:loads(pt))
nodes_data=nodes_data.set_index('osm_id',drop=False)
nodes_data=gpd.GeoDataFrame(nodes_data,geometry='geometry',crs='epsg:4326')
nodes_data['old_geometry']=nodes_data['geometry'].copy()
crs=nodes_data.estimate_utm_crs()
nodes_data=nodes_data.to_crs(crs)
nodes_data['depth']=nodes_data['contour_line_id'].apply(lambda contour_line_id:G.nodes()[contour_line_id]['depth'])
t2=time.time()
print(t2-t1)

0.49356961250305176


In [5]:
len(nodes_data)

17433

In [6]:
max_nb_neighbors=float('inf')
t1=time.time()
nodes_data['elevation']=None
contour_line_ids=nodes_data['contour_line_id'].unique()
contour_lines_ids=sorted(contour_line_ids,key=lambda contour_line_id:len(list(G.neighbors(contour_line_id))))
contour_lines_ids=contour_lines_ids[::-1]
print('sorted')
for k,root_contour_line_id in enumerate(contour_lines_ids):
    if (k+1)%10==0:
        print('%i/%i'%(k+1,len(contour_line_ids)))
    neighbor_contour_line_ids=list(G.neighbors(root_contour_line_id))
    print('%i neighbors'%len(neighbor_contour_line_ids))
    if 0<len(neighbor_contour_line_ids)<max_nb_neighbors:
        t3=time.time()
        local_contours_line_ids=[root_contour_line_id]+neighbor_contour_line_ids
        local_contour_data=get_nodes_data(cursor,local_contours_line_ids)
        local_contour_data=local_contour_data.to_crs(crs)
        local_contour_data=local_contour_data.set_index('id')
        t4=time.time()
        print('setting up df took %f'%(t4-t3))
        local_nodes_data=nodes_data[nodes_data.contour_line_id==root_contour_line_id].copy()
        local_nodes_data['elevation']=local_nodes_data['geometry'].apply(lambda osm_pt:get_osm_node_elevation(osm_pt,local_contour_data))
        nodes_data.loc[nodes_data.contour_line_id==root_contour_line_id,'elevation']=local_nodes_data.loc[:,'elevation']
    
        t5=time.time()
        print('finding affine basis took %f'%(t5-t4))
        print('intermediate time %f'%(t5-t1))
t2=time.time()
print(t2-t1)

sorted
607 neighbors
setting up df took 0.691638


/home/pierre/anaconda3/conda/envs/pygmt/lib/python3.11/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


finding affine basis took 34.247826
intermediate time 34.950515
228 neighbors
setting up df took 0.813902


/home/pierre/anaconda3/conda/envs/pygmt/lib/python3.11/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


finding affine basis took 31.999556
intermediate time 67.764047
205 neighbors
setting up df took 0.876482


...

/home/pierre/anaconda3/conda/envs/pygmt/lib/python3.11/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


finding affine basis took 0.005715
intermediate time 643.396146
1 neighbors
setting up df took 0.018191
finding affine basis took 0.004454
intermediate time 643.418849
1 neighbors
setting up df took 0.022032
finding affine basis took 0.004707
intermediate time 643.445630
1 neighbors
setting up df took 0.011796
finding affine basis took 0.004486
intermediate time 643.461967
1 neighbors
setting up df took 0.034767
finding affine basis took 0.010045
intermediate time 643.506850
810/971
1 neighbors
setting up df took 0.020597
finding affine basis took 0.004779
intermediate time 643.532274
1 neighbors
setting up df took 0.012962
finding affine basis took 0.005911
intermediate time 643.551188
1 neighbors
setting up df took 0.011225
finding affine basis took 0.003448
intermediate time 643.565900
1 neighbors
setting up df took 0.010631
finding affine basis took 0.004179
intermediate time 643.580739
1 neighbors
setting up df took 0.009644
finding affine basis took 0.003603
intermediate time 643

/home/pierre/anaconda3/conda/envs/pygmt/lib/python3.11/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/home/pierre/anaconda3/conda/envs/pygmt/lib/python3.11/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/home/pierre/anaconda3/conda/envs/pygmt/lib/python3.11/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


setting up df took 0.046727
finding affine basis took 0.004775
intermediate time 643.645573
1 neighbors
setting up df took 0.045346
finding affine basis took 0.004129
intermediate time 643.695076
1 neighbors
setting up df took 0.005417
finding affine basis took 0.002702
intermediate time 643.703221
1 neighbors
setting up df took 0.004938
finding affine basis took 0.005175
intermediate time 643.713359
1 neighbors
setting up df took 0.006066
finding affine basis took 0.003160
intermediate time 643.722620
820/971
1 neighbors
setting up df took 0.005691
finding affine basis took 0.003862
intermediate time 643.732208
1 neighbors
setting up df took 0.005257
finding affine basis took 0.003364
intermediate time 643.740857
1 neighbors
setting up df took 0.006745
finding affine basis took 0.004003
intermediate time 643.751638
1 neighbors
setting up df took 0.005433
finding affine basis took 0.003371
intermediate time 643.760484
1 neighbors
setting up df took 0.006470
finding affine basis took 0.

/home/pierre/anaconda3/conda/envs/pygmt/lib/python3.11/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


In [7]:
nodes_data=nodes_data[nodes_data.elevation==nodes_data.elevation]
diffs=nodes_data['elevation']-nodes_data['contour_line_id'].apply(lambda node:G.nodes()[node]['elevation'])
np.min(diffs),np.mean(diffs),np.median(diffs),np.max(diffs)
                                                                  

(-0.4596825749848179, 2.514590478087124, 2.5335205308406734, 4.999915106494939)

In [8]:
UPDATE_ELEVATION=True
if UPDATE_ELEVATION:
    t1=time.time()
    cmd="""CREATE TEMPORARY TABLE temp (
    `osm_id` BIGINT NOT NULL,
    `elevation` FLOAT NOT NULL

    );"""
    cursor.execute(cmd)

    cmd="INSERT INTO temp(`osm_id`,`elevation`) VALUES %s"%',\n'.join(['(%i,%f)'%(osm_id,row['elevation']) for osm_id,row in nodes_data.iterrows()])
    cursor.execute(cmd)

    cmd=""" UPDATE %s JOIN temp
    ON %s.osm_id=temp.osm_id
    SET %s.elevation=temp.elevation

    ;"""%(osm_nodes_table_name,osm_nodes_table_name,osm_nodes_table_name)
    cursor.execute(cmd)
    
    cmd="DROP TABLE temp"
    cursor.execute(cmd)
    t2=time.time()
    print(t2-t1)
2

1.2758359909057617


2